# Intro dlt -> LanceDB loading example

https://lu.ma/cnpdoc5n

If you want to play around with this notebook and make edits in future, we highly recommend making a copy since the link is view only! Also make sure you're signed in with your Google account to be able to add secrets.

Before going into a more complex example, we will go through a simple example of how to load the course Q&A data into LanceDB.

## Install requirements

To create a json -> lancedb pipeline, we need to install:
1. dlt with lancedb extras
2. sentence-transformers: we need to use an embedding model to vectorize and store data inside LanceDB. For this we choose the open-source model "sentence-transformers/all-MiniLM-L6-v2".

In [10]:
%%capture
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers

## Load the data

We'll first load the data just into LanceDB, without embedding it. LanceDB stores both the data and the embeddings, and can also embed data and queries on the fly.

Some definitions:
* A dlt **source** is a grouping of **resources** (e.g. all your data from Hubspot)
* A dlt **resource** is a function that yields data (e.g. a function yielding all your Hubspot companies)
* A dlt **pipeline** is how you ingest your data

Loading the data consists of a few steps:
1. Use the requests library to get the data
2. Define a dlt resource that yields the individual documents
3. Create a dlt pipeline and run it

In [11]:
!rm -R -d ./.lancedb

rm: cannot remove './.lancedb': No such file or directory


In [12]:
import requests
import dlt

qa_dataset = requests.get("https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1").json()

@dlt.resource
def qa_documents():
  for course in qa_dataset:
    yield course["documents"]

pipeline = dlt.pipeline(pipeline_name="from_json", destination="lancedb", dataset_name="qanda")

load_info = pipeline.run(qa_documents, table_name="documents")

print(load_info)

documents
[{'name': 'text', 'data_type': 'text', 'nullable': True}, {'name': 'section', 'data_type': 'text', 'nullable': True}, {'name': 'question', 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
_dlt_loads
[{'name': 'load_id', 'data_type': 'text', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': True}, {'name': 'status', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_version_hash', 'data_type': 'text', 'nullable': True}]
_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type': 'text', 'nullable': False}, {'name': 'created_at', 'data_type': 'timestamp'

In [13]:
import lancedb

db = lancedb.connect("./.lancedb")
print(db.table_names())

['qanda____dlt_loads', 'qanda____dlt_pipeline_state', 'qanda____dlt_version', 'qanda___dltSentinelTable', 'qanda___documents']


In [14]:
db_table = db.open_table("qanda___documents")
db_table.to_pandas()

,id__,text,section,question,_dlt_load_id,_dlt_id
0,c7cf7c0b-aa4d-5cba-873f-5363fb4c110d,The purpose of this document is to capture fre...,General course-related questions,Course - When will the course start?,1722929652.513778,aBn3negHYe0uNg
1,9ec184a4-fdf1-51a1-a681-ecd857f8367f,GitHub - DataTalksClub data-engineering-zoomca...,General course-related questions,Course - What are the prerequisites for this c...,1722929652.513778,Rb4yczk5fdwqOQ
2,18e04d22-a7ee-5e8f-8ff0-cbb2ec4d2d88,"Yes, even if you don't register, you're still ...",General course-related questions,Course - Can I still join the course after the...,1722929652.513778,KRa56SKvqR2m3g
3,546a1915-0b23-5742-8779-942b3d6dc99f,You don't need it. You're accepted. You can al...,General course-related questions,Course - I have registered for the Data Engine...,1722929652.513778,zezrRZFCJKozoA
4,95f8aded-6e9f-5ef1-900e-3762420c2388,You can start by installing and setting up all...,General course-related questions,Course - What can I do before the course starts?,1722929652.513778,RJ2X0L98dK4PTw
...,...,...,...,...,...,...
943,bf34682c-87d7-50ea-95af-bcb133e176ce,Problem description\nThis is the step in the c...,Module 6: Best practices,Github actions: Permission denied error when e...,1722929652.513778,/2yNN566vpJO+g
944,713cbaa6-8823-5b31-8df3-ca4ef50d8c90,Problem description\nWhen a docker-compose fil...,Module 6: Best practices,Managing Multiple Docker Containers with docke...,1722929652.513778,Dn6L2pOkcHTjJQ
945,ed88c06b-4408-56ae-93f5-645353c6fa34,Problem description\nIf you are having problem...,Module 6: Best practices,AWS regions need to match docker-compose,1722929652.513778,o61E3DRfGsBtlw
946,48fbb6db-ef08-50ff-9d71-60e979924f86,Problem description\nPre-commit command was fa...,Module 6: Best practices,Isort Pre-commit,1722929652.513778,M/oUuu0ospDjvQ


## Load and embed the data

Now we load the same data again (into a new table), but embed it directly with the `lancedb_adapter`. This consists of the following steps:

1. Define the embedding model to use via ENV variables
2. Define a new pipeline to load the same data and embed the "text" and "question" columns with the `lancedb_adapter`

You can use any embedding model, from open source to OpenAI. We've chosen the [`all-MiniLM-L6-v2`](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) sentence transformer for speed and simplicty.

Note: this pipeline runs slightly longer because it has to download the model and embed the data.

In [15]:
import os
from dlt.destinations.adapters import lancedb_adapter

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

pipeline = dlt.pipeline(pipeline_name="from_json_embedded", destination="lancedb", dataset_name="qanda_embedded")

load_info = pipeline.run(lancedb_adapter(qa_documents, embed=["text", "question"]), table_name="documents")

documents
[{'name': 'text', 'x-lancedb-embed': True, 'data_type': 'text', 'nullable': True}, {'name': 'section', 'data_type': 'text', 'nullable': True}, {'name': 'question', 'x-lancedb-embed': True, 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

_dlt_loads
[{'name': 'load_id', 'data_type': 'text', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': True}, {'name': 'status', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_version_hash', 'data_type': 'text', 'nullable': True}]
_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type': 'text', 'nullable': False}, {'name': 'created_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
_dlt_version
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version'

In [16]:
db = lancedb.connect("./.lancedb")

In [17]:
db_table = db.open_table("qanda_embedded___documents")

That's all for this intro example! The DB could now be used as a basis for a RAG.

# Create an up-to-date RAG with dlt and LanceDB

**Note on running this notebook**: We are going to download and use a local Ollama instance for the RAG, so preferably select the **T4 GPU** in the runtime when starting this notebook (Runtime > Change runtime type > Hardware accelerator > T4 GPU).

You can also use the default CPU in case you're facing technical issues, but then your LLM responses might be slower (~2 mins/response)

## Part 1: Create a Notion -> LanceDB pipeline using dlt

### 1. Install requirements

To create a notion -> lancedb pipeline, we need to install:
1. dlt with lancedb extras
2. sentence-transformers: we need to use an embedding model to vectorize and store data inside LanceDB. For this we choose the open-source model "sentence-transformers/all-MiniLM-L6-v2".

In [5]:
%%capture
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers
!yes | dlt init rest_api lancedb

### 2. Create a dlt project with rest_api source and lancedb destination

We now create a dlt project using the command `dlt init <source> <destination>`.

### 3. Add API credentials

We are going to be using option 2. It's not advisable to paste sensitive information like API keys inside the code, so instead we're going to include them inside the secrets tab in the side panel of the notebook. This will allow us to access the secret values from the notebook.

Since we are using the OSS version of LanceDB and OSS embedding models, we only need to specify the API key for Notion.

**Note**: You will need to copy the [notion API key](https://share.1password.com/s#da9KgMwPaZUaey3WCaD7ICJoyHDGd3Xos2EZ29WrSWQ) into the secrets tab under the name `SOURCES__REST_API__NOTION__API_KEY`. Make sure to enable notebook access after pasting the key.

In [9]:
import os
from google.colab import userdata

os.environ["SOURCES__REST_API__NOTION__API_KEY"] = userdata.get("SOURCES__REST_API__NOTION__API_KEY")

os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL_PROVIDER"] = "sentence-transformers"
os.environ["DESTINATION__LANCEDB__EMBEDDING_MODEL"] = "all-MiniLM-L6-v2"

os.environ["DESTINATION__LANCEDB__CREDENTIALS__URI"] = ".lancedb"

### 4. Write the pipeline code

**Note**: We first go over the code step by step before putting it into runnable cells

1. Import necessary modules (run this cell)

In [18]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

### 5. Run the pipeline

Run this block:

In [19]:
from datetime import datetime, timezone

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="employee_handbook")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "workshop",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config,name="employee_handbook")

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="employee_handbook",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    # last_value = last_edited_time.last_value
    # print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block

def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="employee_handbook",
        write_disposition="merge"
    )
    print(load_info)

load_notion()

_dlt_loads
[{'name': 'load_id', 'data_type': 'text', 'nullable': False}, {'name': 'schema_name', 'data_type': 'text', 'nullable': True}, {'name': 'status', 'data_type': 'bigint', 'nullable': False}, {'name': 'inserted_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'schema_version_hash', 'data_type': 'text', 'nullable': True}]
_dlt_pipeline_state
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version', 'data_type': 'bigint', 'nullable': False}, {'name': 'pipeline_name', 'data_type': 'text', 'nullable': False}, {'name': 'state', 'data_type': 'text', 'nullable': False}, {'name': 'created_at', 'data_type': 'timestamp', 'nullable': False}, {'name': 'version_hash', 'data_type': 'text', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
_dlt_version
[{'name': 'version', 'data_type': 'bigint', 'nullable': False}, {'name': 'engine_version'

### 6. Visualize the output

In [20]:
import lancedb

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___employee_handbook")

dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,71e89a85-ae0b-5b68-866b-bd3922ec7548,"[-0.05858811, -0.07540446, 0.033775203, 0.0096...",c0262981-b5f1-4a57-a91f-2e75f649b86c,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-18 14:00:00+00:00,2024-08-06 07:36:01.494461+00:00,1722929760.0091553,h1qf2yOVuz48Ug
1,a28e913f-761f-5684-8cd5-0d0c49e0338c,"[-0.004968941, -0.003911972, 0.028705625, 0.00...",faacf4ec-90be-4e96-b8b9-29b5112bc7ca,paragraph,Employees receive [20 days] of Paid Time Off (...,2024-06-26 09:03:00+00:00,2024-08-06 07:36:01.500401+00:00,1722929760.0091553,Ywda5NRjoQX9bQ
2,a18932d9-1583-5c42-bd0d-0f96738c5e6c,"[0.032060888, 0.024244698, 0.008471344, 0.0317...",e6021a51-f403-4950-80c2-ebff005c7289,paragraph,Our company observes the following holidays: N...,2024-06-26 09:08:00+00:00,2024-08-06 07:36:01.500731+00:00,1722929760.0091553,2PfXlm0MpY7gnQ
3,93661874-13a2-5a43-bed8-868005dfd5e2,"[-0.0131553095, 0.008382407, 0.017044391, 0.05...",b8f4cc6d-c28c-4071-9545-caadce5eb37b,paragraph,These holidays are considered “off-days” for m...,2024-06-26 09:09:00+00:00,2024-08-06 07:36:01.501002+00:00,1722929760.0091553,Gdh4l3IBzRt3Kw
4,b220778f-1118-5c22-b614-3bc0fd0a602b,"[0.027987516, 0.067343615, 0.03980646, 0.00774...",ea7a1beb-6874-4f41-966d-dc1f80a1f635,paragraph,Employees who are unable to work due to illnes...,2024-06-26 09:11:00+00:00,2024-08-06 07:36:01.501270+00:00,1722929760.0091553,qCA1hPuul5WmeA
5,d0f801ba-d3cc-5252-ad6e-3285662b609c,"[0.03252615, 0.008159476, 0.08443566, 0.055641...",bd7a9110-fac5-4270-9493-4039ca67b467,paragraph,Losing a loved one is traumatizing. If this ha...,2024-06-26 09:17:00+00:00,2024-08-06 07:36:01.501560+00:00,1722929760.0091553,jRRfNLf12i9Y3A
6,579b97f9-a5e2-53af-b4f7-efc9ad5105ad,"[-0.007314052, 0.014710642, -0.019091193, 0.02...",b1718dee-8c0f-4189-8c75-0e8c7844a501,paragraph,"In accordance with German law, we offer a comp...",2024-06-26 09:20:00+00:00,2024-08-06 07:36:01.501848+00:00,1722929760.0091553,BgDtWFQUnof1Lw
7,a9083b7e-22cc-5b1f-8040-cb7aa1f72338,"[-0.031538427, 0.0342599, -0.027282646, 0.0275...",5bfa90c5-461d-406a-9324-a1dd54bad0d5,paragraph,We recognize the vital role that fathers and p...,2024-06-26 09:21:00+00:00,2024-08-06 07:36:01.502138+00:00,1722929760.0091553,Na5Rhr+YugCblQ
8,6adeb540-d180-5d40-bc84-c40e5c173ea1,"[-0.03892389, 0.1208173, 0.046208583, -0.00543...",baac0ba4-9b60-450e-8cc1-1e6e2a0fb7d9,paragraph,"In this section, we describe what we offer to ...",2024-07-03 17:34:00+00:00,2024-08-06 07:36:01.832842+00:00,1722929760.0091553,s92RGXeOSEVZmQ
9,cffdb1bb-a146-5e90-8fbb-a1d577a2a98e,"[-0.07571499, 0.14543605, 0.0011521844, -0.024...",0e429073-6383-4918-8961-fcc66346067f,paragraph,{edited} Employee health is important to us. W...,2024-07-18 17:28:00+00:00,2024-08-06 07:36:01.833270+00:00,1722929760.0091553,fqw93jBCsyImfQ


 ---

Now we make change to one of the paragraphs and run the pipeline again to see the effect of incremental loading. We observe two things:
1. The column `inserted_at_time` only changed for the updated row, implying that only this row was added
2. Looking at the primary key `block_id` we see that the original row was dropped and the updated row was inserted

## Part 2: Create a RAG bot using Ollama

With the contents from the employee handbook vectorized and stored in LanceDB, we're now ready to create our RAG with Ollama.


1. Install Ollama into the notebook's local runtime

In [21]:
%%capture

!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > nohup.out 2>&1 &
!ollama pull llama2-uncensored

In [22]:
!pip install ollama

In [23]:
import ollama

In [24]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):
    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

In [25]:
#Workshop task


from datetime import datetime, timezone
import dlt

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response):
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request):
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="employee_handbook")
def rest_api_notion_resource():
    notion_config = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": dlt.secrets["sources.rest_api.notion.api_key"]
            },
            "headers": {
                "Content-Type": "application/json",
                "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "homework",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config, name="employee_handbook")

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    content = ""

    if block_type == "paragraph":
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            pass

    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="homework",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time": {"dedup_sort": "desc"}}
)
def rest_api_notion_incremental(
    last_edited_time=dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z", primary_key=("block_id"))
):
    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not block["content"]:
            continue
        yield block

def load_notion():
    pipeline = dlt.pipeline(
        pipeline_name="company_policies_hw",
        destination="lancedb",
        dataset_name="notion_pages",
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="homework",
        write_disposition="merge"
    )
    print(load_info)


In [26]:
load_notion()

string
load_id
string
schema_name
int64
status
timestamp[us, tz=UTC]
inserted_at
string
schema_version_hash
_dlt_loads
[]
int64
version
int64
engine_version
string
pipeline_name
string
state
timestamp[us, tz=UTC]
created_at
string
version_hash
string
_dlt_load_id
string
_dlt_id
_dlt_pipeline_state
[]
int64
version
int64
engine_version
timestamp[us, tz=UTC]
inserted_at
string
schema_name
string
version_hash
string
schema
_dlt_version
[]
homework
[{'name': 'block_id', 'nullable': False, 'primary_key': True, 'data_type': 'text'}, {'name': 'block_type', 'data_type': 'text', 'nullable': True}, {'name': 'content', 'x-lancedb-embed': True, 'data_type': 'text', 'nullable': True}, {'dedup_sort': 'desc', 'name': 'last_edited_time', 'data_type': 'timestamp', 'nullable': True}, {'name': 'inserted_at_time', 'data_type': 'timestamp', 'nullable': True}, {'name': '_dlt_load_id', 'data_type': 'text', 'nullable': False}, {'name': '_dlt_id', 'data_type': 'text', 'nullable': False, 'unique': True}]
UPLOAD

In [27]:
home="notion_pages___homework"
db = lancedb.connect(".lancedb")
db.table_names()


['notion_pages____dlt_loads',
 'notion_pages____dlt_pipeline_state',
 'notion_pages____dlt_version',
 'notion_pages___dltSentinelTable',
 'notion_pages___employee_handbook',
 'notion_pages___homework',
 'qanda____dlt_loads',
 'qanda____dlt_pipeline_state',
 'qanda____dlt_version',
 'qanda___dltSentinelTable',
 'qanda___documents',
 'qanda_embedded____dlt_loads',
 'qanda_embedded____dlt_pipeline_state',
 'qanda_embedded____dlt_version',
 'qanda_embedded___dltSentinelTable',
 'qanda_embedded___documents']

In [29]:
table = db.open_table(home)
pd = table.to_pandas()
pd.shape
print(f'Q1) How many rows does the lancedb table "notion_pages__homework" have? ', pd.shape[0])

Q1) How many rows does the lancedb table "notion_pages__homework" have?  17


In [31]:
pd.columns

Index(['id__', 'vector__', 'block_id', 'block_type', 'content',
       'last_edited_time', 'inserted_at_time', '_dlt_load_id', '_dlt_id'],
      dtype='object')

In [32]:
print(
    f"Q2) What value does it store after you've run your pipeline once? ",
    pd.last_edited_time.max(),
)

Q2) What value does it store after you've run your pipeline once?  2024-07-05 23:33:00+00:00


In [33]:
def rag(question, table="notion_pages___employee_handbook", ):
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table(table)

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  # Retrieve the relevant paragraphs on the question
  context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

  # Create a user prompt using the question and retrieved context
  messages.append(
      {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
  )

  # Get the response from the LLM
  response = ollama.chat(
      model="llama2-uncensored",
      messages=messages
  )
  response_content = response['message']['content']
  print(f"Assistant: {response_content}")


In [34]:
# Q3
rag("how many PTO days are the employees entitled to in a year?")

Assistant: Based on the provided context, it appears that employees at this company receive 20 days of Paid Time Off (PTO) per year. Employees can use their PTO beginning in their first week with the company and they accrue one additional day each year after their first year with a cap of 25 total days. If you want to take your leave, you must send a request through HRIS. Your manager or HR approves your request. You are not required to specify a reason for requesting PTO and cannot transfer remaining PTO to the next year.
If you decide to use your PTO time, you may be compensated with accrued leave when leaving the company according to local law. For non-exempt employees who work on holidays, they will receive their regular hourly rate plus a premium for working that day. Exempt employees will receive an additional day of PTO within 12 months after the holiday and hours worked during that time period count towards overtime pay eligibility.
